# Chapter 05: AI Agent Workflows

In our previous discussions, we explored the utility of structured outputs and AI agents in the context of chat applications. Today, we take a leap forward by combining these foundational elements with workflow orchestration, leveraging LLMs. This enables us to build highly sophisticated workflows, integrating multiple steps and diverse components such as functions, data transformations, machine learning models, vision models, and more. The result is an output in a predefined structure that can be integrated into business systems like databases, ERP systems, and other organizational infrastructure.  

The integration of LLMs into these workflows offers a unique advantage: the ability to handle unstructured data and convert it into structured formats that can be effortlessly pushed into various systems. This capability addresses a pivotal challenge faced by businesses today - efficiently managing and utilizing the vast amounts of unstructured data generated daily.  

Moreover, these workflows can be automated to react to specific triggers, such as the addition of a new object in a system or the receipt of an alert. Such automation not only enhances operational efficiency but also ensures timely and accurate decision-making. The output generated from these workflows can either be written directly into the systems or subjected to a human-in-the-loop approval process, ensuring the perfect balance between automation and human oversight.

In [1]:
from __future__ import annotations
import sys
from typing import Any, Callable, Literal
from functools import reduce
from pydantic import BaseModel, Field
from loguru import logger
from language_models.agent import Agent, OutputType, PromptingStrategy
from language_models.models.llm import OpenAILanguageModel
from language_models.proxy_client import ProxyClient
from language_models.settings import settings

In [2]:
logger.remove()
logger.add(sys.stderr, format="{message}", level="INFO")

proxy_client = ProxyClient(
    client_id=settings.CLIENT_ID,
    client_secret=settings.CLIENT_SECRET,
    auth_url=settings.AUTH_URL,
    api_base=settings.API_BASE,
)

In [3]:
llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model="gpt-4",
    max_tokens=250,
    temperature=0.2,
)

In our single workflow design, we have integrated distinct components:
- **`WorkflowFunctionStep`:** Manages specific function invocations.
- **`WorkflowAgentStep`:** Leverages AI agents for tasks.
- **`WorkflowTransformationStep`:** Focuses on data transformations, offering mechanisms like map, filter, and reduce to efficiently process and restructure data.

In [4]:
class WorkflowStepOutput(BaseModel):
    """Class that represents the output of a step."""

    inputs: dict[str, Any]
    output: (
        str
        | int
        | float
        | dict[str, Any]
        | BaseModel
        | list[str]
        | list[int]
        | list[float]
        | list[dict[str, Any]]
        | list[BaseModel]
        | None
    )

class WorkflowFunctionStep(BaseModel):
    """Class that implements a function step.

    Attributes:
        name: The name of the step.
        inputs: The Pydantic model that represents the input arguments.
        function: The function that will be invoked when calling this step.
    """

    name: str
    inputs: type[BaseModel]
    function: Callable[[Any], Any]

    def invoke(self, inputs: dict[str, Any], verbose: bool) -> WorkflowStepOutput:
        inputs = {key: value for key, value in inputs.items() if key in self.inputs.model_fields}
        inputs = self.inputs.model_validate(inputs).model_dump()
        if verbose:
            logger.opt(colors=True).info(f"<b><fg #EC9A3C>Function Input</fg #EC9A3C></b>: {inputs}")

        output = self.function(**inputs)
        if verbose:
            logger.opt(colors=True).info(f"<b><fg #EC9A3C>Function Output</fg #EC9A3C></b>: {output}")

        return WorkflowStepOutput(inputs=inputs, output=output)

class WorkflowAgentStep(BaseModel):
    """Class that implements an agent step.

    Attributes:
        name: The name of the step.
        agent: The agent that will be invoked when calling this step.
    """

    name: str
    agent: Agent

    def invoke(self, inputs: dict[str, Any], verbose: bool) -> WorkflowStepOutput:
        inputs = {variable: inputs.get(variable) for variable in self.agent.prompt_variables}
        if verbose:
            logger.opt(colors=True).info(f"<b><fg #EC9A3C>Agent Input</fg #EC9A3C></b>: {inputs}")

        output = self.agent.invoke(inputs)
        if verbose:
            logger.opt(colors=True).info(f"<b><fg #EC9A3C>Agent Output</fg #EC9A3C></b>: {output.final_answer}")

        return WorkflowStepOutput(inputs=inputs, output=output.final_answer)

class WorkflowTransformationStep(BaseModel):
    """Class that implements a transformation step.

    Attributes:
        name: The name of the step.
        input_field: The name of the field values to transform.
        transformation: The transformation to apply (can be map, filter, reduce).
        function: The function used for the transformation.
    """

    name: str
    input_field: str
    transformation: Literal["map", "filter", "reduce"]
    function: Callable[[Any], Any]

    def invoke(self, inputs: dict[str, Any], verbose: bool) -> WorkflowStepOutput:
        values = inputs[self.input_field]
        inputs = {self.input_field: values}
        if verbose:
            logger.opt(colors=True).info(f"<b><fg #EC9A3C>Transformation Input</fg #EC9A3C></b>: {inputs}")

        if self.transformation == "map":
            transformed_values = map(self.function, values)
            output = list(transformed_values) if isinstance(values, list) else dict(transformed_values)
        elif self.transformation == "filter":
            transformed_values = filter(self.function, values)
            output = list(transformed_values) if isinstance(values, list) else dict(transformed_values)
        else:
            output = reduce(self.function, values)

        if verbose:
            logger.opt(colors=True).info(f"<b><fg #EC9A3C>Transformation Output</fg #EC9A3C></b>: {output}")

        return WorkflowStepOutput(inputs=inputs, output=output)

Next, we introduce several components to execute a workflow comprising various steps.

In [5]:
class WorkflowStateManager(BaseModel):
    """Class that implements a state manager."""

    state: dict[str, Any]

    def update(self, name: str, step: WorkflowStepOutput) -> None:
        """Updates the state values."""
        self.state[name] = step.output

class WorkflowOutput(BaseModel):
    """Class that represents the workflow output."""

    inputs: dict[str, Any]
    output: (
        str
        | int
        | float
        | dict[str, Any]
        | BaseModel
        | list[str]
        | list[int]
        | list[float]
        | list[dict[str, Any]]
        | list[BaseModel]
        | None
    )

class Workflow(BaseModel):
    """Class that implements a workflow.

    Attributes:
        name: The name of the workflow.
        description: The description of what the workflow does.
        steps: The steps of the workflow.
        inputs: The workflow inputs.
        output: The name of the step value to output.
    """

    name: str
    description: str
    steps: list[WorkflowAgentStep | WorkflowFunctionStep | WorkflowTransformationStep]
    inputs: type[BaseModel]
    output: str
    verbose: bool

    def invoke(self, inputs: dict[str, Any]) -> WorkflowOutput:
        """Runs the workflow."""
        inputs = self.inputs.model_validate(inputs).model_dump()
        state_manager = WorkflowStateManager(state=inputs)
        for step in self.steps:
            if self.verbose:
                logger.opt(colors=True).info(f"<b><fg #2D72D2>Running Step</fg #2D72D2></b>: {step.name}")

            output = step.invoke(state_manager.state, self.verbose)
            state_manager.update(step.name, output)

        output = state_manager.state.get(self.output)
        if self.verbose:
            logger.opt(colors=True).success(f"<b><fg #32A467>Workflow Output</fg #32A467></b>: {output}")

        return WorkflowOutput(inputs=inputs, output=output)

In this scenario, we created an example workflow designed to extract and process numbers from a given text. The workflow begins with an agent step, where an AI agent is tasked with extracting all numbers from the user's input text. This is followed by a function step that sorts the extracted numbers in ascending order. Finally, a transformation step filters the sorted numbers, retaining only those greater than 10. 

In [6]:
system_prompt = """You are an AI assistant designed to help users with a variety of tasks.

Extract all numbers from the user's input text."""

agent = Agent.create(
    llm=llm,
    system_prompt=system_prompt,
    prompt="{prompt}",
    prompt_variables=["prompt"],
    output_type=OutputType.ARRAY_INTEGER,
    prompting_strategy=PromptingStrategy.SINGLE_COMPLETION,
    verbose=True,
)

agent_step = WorkflowAgentStep(name="numbers", agent=agent)

class Function(BaseModel):
    numbers: list[int]

function_step = WorkflowFunctionStep(name="sort", inputs=Function, function=lambda numbers: sorted(numbers))

filter_step = WorkflowTransformationStep(name="numbers_greater_10", input_field="sort", transformation="filter", function=lambda number: number > 10)

In [7]:
class Prompt(BaseModel):
    prompt: str = Field(description="The user prompt")

workflow = Workflow(
    name="Find numbers greater than 10",
    description="Extracts numbers from a given text",
    steps=[agent_step, function_step, filter_step],
    inputs=Prompt,
    output="numbers_greater_10",
    verbose=True,
)

The workflow is driven by a prompt containing a narrative about a hiking trip. Upon execution, the workflow seamlessly processes the input, notably extracting key numerical data points and refining them as specified. These workflows will be more useful later on.

In [8]:
prompt = """Last weekend, six of us went on a 15-kilometer hike, starting at 7 AM.

By noon, we had covered 10 kilometers and reached Mount Elbert's 4,401-meter summit by 2 PM, with a temperature of 12°C.

We camped 5 kilometers away by 6 PM with 12 others and returned home by 5 PM the next day."""

output = workflow.invoke({"prompt": prompt})

Running Step: numbers
Agent Input: {'prompt': "Last weekend, six of us went on a 15-kilometer hike, starting at 7 AM.\n\nBy noon, we had covered 10 kilometers and reached Mount Elbert's 4,401-meter summit by 2 PM, with a temperature of 12°C.\n\nWe camped 5 kilometers away by 6 PM with 12 others and returned home by 5 PM the next day."}
Final Answer: [6, 15, 7, 10, 4401, 2, 12, 5, 6, 12, 5]
Agent Output: [6, 15, 7, 10, 4401, 2, 12, 5, 6, 12, 5]
Running Step: sort
Function Input: {'numbers': [6, 15, 7, 10, 4401, 2, 12, 5, 6, 12, 5]}
Function Output: [2, 5, 5, 6, 6, 7, 10, 12, 12, 15, 4401]
Running Step: numbers_greater_10
Transformation Input: {'sort': [2, 5, 5, 6, 6, 7, 10, 12, 12, 15, 4401]}
Transformation Output: [12, 12, 15, 4401]
Workflow Output: [12, 12, 15, 4401]


In [9]:
print(output.output)

[12, 12, 15, 4401]
